In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [3]:
company=pd.read_csv("A:/Data Science/assignments/Random forest/Company_Data.csv")

In [11]:
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,group
0,9.50,138,73,11,276,120,42,17,High
1,11.22,111,48,16,260,83,65,10,High
2,10.06,113,35,10,269,80,59,12,High
3,7.40,117,100,4,466,97,55,14,less
4,4.15,141,64,3,340,128,38,13,less


In [5]:
group=np.where(company['Sales']>=8,'High','less')
company['group']=group

In [6]:
company.group.value_counts()

less    236
High    164
Name: group, dtype: int64

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
lab=LabelEncoder()
shelf=lab.fit_transform(company.ShelveLoc)
urb=lab.fit_transform(company.Urban)
country=lab.fit_transform(company.US)

In [10]:
del company['ShelveLoc']
del company['Urban']
del company['US']

In [12]:
company['Shelfloc']=shelf
company['Urban']=urb
company['US']=country

In [18]:
company.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,group,Shelfloc,Urban,US
0,9.50,138,73,11,276,120,42,17,High,0,1,1
1,11.22,111,48,16,260,83,65,10,High,1,1,1
2,10.06,113,35,10,269,80,59,12,High,2,1,1
3,7.40,117,100,4,466,97,55,14,less,2,1,1
4,4.15,141,64,3,340,128,38,13,less,0,1,0


In [14]:
company.columns

Index(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population', 'Price',
       'Age', 'Education', 'group', 'Shelfloc', 'Urban', 'US'],
      dtype='object')

In [17]:
company.dtypes

Sales          float64
CompPrice        int64
Income           int64
Advertising      int64
Population       int64
Price            int64
Age              int64
Education        int64
group           object
Shelfloc        object
Urban           object
US              object
dtype: object

In [16]:
company.Shelfloc=company.Shelfloc.astype(object)
company.US=company.US.astype(object)
company.Urban=company.Urban.astype(object)

In [19]:
X=company[[ 'CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age',
       'Education','Shelfloc', 'Urban', 'US']]
y=company[['group']]

In [24]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [61]:
kf=KFold(n_splits=10,shuffle=False)
for train_index ,test_index in kf.split(X,y):
    X_train,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]
    bag=BaggingClassifier(base_estimator=RandomForestClassifier(criterion='entropy',n_estimators=50,oob_score=True,min_samples_split=10,max_features=8,n_jobs=3),n_estimators=10)
    bag.fit(X_train,y_train)
    train=np.mean(bag.score(X_train,y_train))
    test=np.mean(bag.score(X_test,y_test))


In [62]:
train

0.9555555555555556

In [63]:
test

0.9

In [66]:
values1=bag.predict(X_test)

In [68]:
pd.crosstab(y_test.group,values1)

col_0,High,less
group,,
High,15,3
less,1,21


In [57]:
#Accuracy of 90%